# Conv2D implementation of the Inner speech dataset

In [1]:
import os
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(23)

warnings.filterwarnings(action = "ignore", category = DeprecationWarning )
warnings.filterwarnings(action = "ignore", category = FutureWarning )

In [2]:
%%time
# The Storage variables with their respected names
X_train =np.load('processed_data/X_train.npy')
Y_train =np.load('processed_data/Y_train.npy')
X_val = np.load('processed_data/X_val.npy')
Y_val =np.load('processed_data/Y_val.npy')

# Printing the shape
print("The shape of the X_train x Y_train ", X_train.shape , " x ",Y_train.shape)
print("The shape of the X_val x Y_val ", X_val.shape , " x ",Y_val.shape)


The shape of the X_train x Y_train  (1799, 128, 512)  x  (1799,)
The shape of the X_val x Y_val  (223, 128, 512)  x  (223,)
CPU times: total: 484 ms
Wall time: 1.42 s


## Applying CNN

In [3]:
import torch
import torch.nn as nn

In [4]:
# Define model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(32 * 64 * 256, 4)  # num_classes is the number of output classes

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)  # Flatten the input
        x = self.fc(x)
        return x

model = CNN()
model

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=524288, out_features=4, bias=True)
)

In [5]:
# Selecting Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print('Using device:', device)

Using device: cuda


In [6]:
# Unsqueezing and Flattening the dataset
X_train_unsq = torch.from_numpy(X_train).unsqueeze(1).float().to(device)   # Add a channel dimension and convert to float tensor
Y_train_flat = torch.from_numpy(Y_train).long().to(device)                 # Convert to long tensor

In [7]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [8]:
batch_size = 32  # Adjust the batch size as per your GPU memory capacity
accumulation_steps = 4  # Accumulate gradients over 4 batches before performing optimizer step
num_batches = len(X_train_unsq) // batch_size
num_epochs =5
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    for i in range(num_batches):
        start = i * batch_size
        end = (i + 1) * batch_size
        inputs = X_train_unsq[start:end].to(device)
        targets = Y_train_flat[start:end].to(device)

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss = loss / accumulation_steps  # Divide the loss by accumulation steps
        loss.backward()

        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

print("Training complete!")

Epoch [1/5], Loss: 4.9132
Epoch [2/5], Loss: 3.0969
Epoch [3/5], Loss: 1.5147
Epoch [4/5], Loss: 0.3857
Epoch [5/5], Loss: 0.4447
Training complete!


In [ ]:
## Training accuracy
# Make prediction
device = torch.device("cpu")
model.to(device)

X_train_acc = torch.from_numpy(X_train).unsqueeze(1).float()[1:200].to(device)   # Add a channel dimension and convert to float tensor
Y_train_acc = torch.from_numpy(Y_train).long()[1:200].to(device)                 # Convert to long tensor               

model.eval()  # Set the model in evaluation mode
with torch.no_grad():
    outputs = model(X_train_acc.to(device))
    _, predicted = torch.max(outputs.data, 1)  # Get the predicted classes

correct = (predicted == Y_train_acc.to(device)).sum().item()
accuracy = correct / len(predicted) * 100
print(f"Accuracy on test data: {accuracy:.2f}%")

## Evaluating the model

In [ ]:
X_val = torch.from_numpy(X_val).unsqueeze(1).float().to(device)   # Add a channel dimension and convert to float tensor
Y_val = torch.from_numpy(Y_val).long().to(device)                 # Convert to long tensor      

model.eval()  # Set the model in evaluation mode
with torch.no_grad():
    outputs = model(X_val.to(device))
    _, predicted = torch.max(outputs.data, 1)  # Get the predicted classes

correct = (predicted == Y_val.to(device)).sum().item()
accuracy = correct / len(predicted) * 100
print(f"Accuracy on test data: {accuracy:.2f}%")

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Y_val.numpy(), predicted.numpy())
classes = [0,1,2,3]

plt.figure(figsize=(10, 8))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)

fmt = '.2f'
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.tight_layout()
plt.show()